In [1]:
from collections import Counter
import numpy as np
import codecs
import re
import jieba

In [2]:
##　准备数据
import jieba
import pandas as pd 

# df_technology
df_technology  = pd.read_csv("./data/technology_news.csv", encoding = 'utf-8')
df_technology  = df_technology.dropna() # 取消可能为空的行数
# sprots
df_sprots   = pd.read_csv("./data/sports_news.csv", encoding = 'utf-8')
df_sprots   = df_sprots.dropna()

#df_society
df_society = pd.read_csv("./data/society_news.csv",encoding = 'utf-8')
df_society = df_society.dropna()

#military
df_military   = pd.read_csv("./data/military_news.csv", encoding = 'utf-8')
df_military   = df_military.dropna()

df_internality = pd.read_csv("./data/international_news.csv",encoding = 'utf-8')
df_international = df_internality.dropna()

df_house = pd.read_csv("./data/house_news.csv",encoding = 'utf-8')
df_house = df_house.dropna()

df_home = pd.read_csv("./data/home_news.csv",encoding = 'utf-8')
df_home = df_home.dropna()

df_finance = pd.read_csv("./data/finance_news.csv",encoding = 'utf-8')
df_finance = df_finance.dropna()

# df_entertainment 
df_entertainment   = pd.read_csv("./data/entertainment_news.csv", encoding = 'utf-8')
df_entertainment   = df_entertainment.dropna()

# df_car
df_car  = pd.read_csv("./data/car_news.csv", encoding = 'utf-8')
df_car  = df_car.dropna()

In [3]:
# 每个种类提取一定数量的文本样本记录
technology     = df_technology.content.values.tolist()
car            = df_car.content.values.tolist()
sprots         = df_sprots.content.values.tolist()
military       = df_military.content.values.tolist()
society        = df_society.content.values.tolist()
international  = df_international.content.values.tolist()
house          = df_house.content.values.tolist()
home           = df_home.content.values.tolist()
finance        = df_finance.content.values.tolist()
entertainment  = df_entertainment.content.values.tolist()

### 额外的添加一些数据

In [48]:
### 从其他地方借来的数据 - from sohu data 
with open('./sohudata/data/sohu_train_car.txt','r',encoding= 'utf-8') as file:
    for line in file.readlines():
        car.append(line)
        
with open('./sohudata/data/sohu_train_house.txt','r',encoding= 'utf-8') as file:
    for line in file.readlines():
        house.append(line)

In [49]:
### 从其他地方借来的数据 - from cnew.train.data
with open("./cnews/cnews_train_sport.txt",'r',encoding= 'utf-8') as file:
    for line in file.readlines():
        sprots.append(line)
        
with open("./cnews/cnews_train_tech.txt",'r',encoding= 'utf-8') as file:
    for line in file.readlines():
        technology.append(line)

In [54]:
with open("./cnews/cnews_train_house.txt",'r',encoding= 'utf-8') as file:
    for line in file.readlines():
        house.append(line)

In [6]:
len(house)

18475

###  1.1统计文本长度 -  文本分类原始数据

In [5]:
AllTexts        = [technology,sprots,society,international,house,home,car,finance,entertainment,military]
AllTexts_length = []
lens = []
for text in AllTexts:
    count = 0
    for oneExample in text:
        if len(oneExample)>=50:
            count = count +1
            lens.append(len(oneExample))
    AllTexts_length.append(count)
#AllTexts_length.sort()
AllTexts_length # 样本长度比较短的是 [military,]

[20064, 24826, 191272, 72353, 11342, 115116, 9186, 113065, 28039, 11849]

In [16]:
label1S      = [5066, 4449, 26795, 4807, 5113, 21534, 5312, 19927, 8508]

In [8]:
label1       = ['科技','体育', '社会', '国际', '房产', '家居', '汽车', '财经','娱乐'] 

In [9]:
label2_sohu  = ['体育', '健康', '女人', '娱乐', '房地产', '教育', '文化', '新闻', '旅游', '汽车', '科技', '财经']  #3000

In [17]:
label3_cnews = ['体育', '娱乐', '家居', '房产', '教育', '时尚', '时政', '游戏', '科技', '财经'] #5000

###  1.1.1统计文本长度 -  sohu数据

In [52]:
file   = "./cnews/cnews_train_house.txt"

In [53]:
writor = open(file,'w',encoding = 'utf-8')
with open('./cnews/cnews.train.txt','r',encoding = 'utf-8') as f:
    for line in f.readlines():
        assert(len(line.split('\t')) == 2)
        items = line.split('\t')
        label = items[0]
        content = items[1]
        if label == '房产':
            writor.write(content)

In [99]:
filetrain = './cnews/cnews.train.txt'
filetest = './cnews/cnews.test.txt'
fileval = './cnews/cnews.val.txt'
files = [filetrain,filetest,fileval]
lens = []
for file in files:
    with open(file,'r',encoding = 'utf-8') as file:
        for line in file.readlines():
            lens.append(len(line))
            
import numpy as np
lenss = np.array(lens)
np.min(lenss),np.mean(lenss),np.max(lenss),len(lenss), np.mean(lenss) + 2*np.std(lenss),np.sum(lenss<600)/len(lenss)*100

(12, 923.50413846153845, 27471, 65000, 2773.3759383087481, 43.150769230769228)

In [98]:
filetrain = './newdata/random/cnews.train.txt'
filetest = './newdata/random/cnews.test.txt'
fileval = './newdata/random/cnews.val.txt'
files = [filetrain,filetest,fileval]
lens = []
for file in files:
    with open(file,'r',encoding = 'utf-8') as file:
        for line in file.readlines():
            lens.append(len(line))
            
import numpy as np
lenss = np.array(lens)
np.min(lenss),np.mean(lenss),np.max(lenss),len(lenss), np.mean(lenss) + 2*np.std(lenss),np.sum(lenss<350)/len(lenss)*100

(142, 554.35139784946239, 36997, 55800, 3443.6430721524366, 86.702508960573482)

In [91]:
filetrain = './sohudata/data/sohu_train.txt'
filetest =  './sohudata/data/sohu_test.txt'
files = [filetrain,filetest,fileval]
lens = []
for file in files:
    with open(file,'r',encoding = 'utf-8') as file:
        for line in file.readlines():
            lens.append(len(line))
            
import numpy as np
lenss = np.array(lens)
np.min(lenss),np.mean(lenss),np.max(lenss),len(lenss), np.mean(lenss) + 2*np.std(lenss),np.sum(lenss<400)/len(lenss)*100

(100, 2606.3519047619047, 88189, 37800, 7965.4533806715181, 5.0238095238095237)

### 1.2手动切分数据

In [27]:
allcontents = []
def Train_Test_Validation_Data_Split(filetrain,filetest,fileval,contents,labels,limits,lens):
    data_train = open(filetrain, 'a', encoding='utf-8')
    data_test  = open(filetest, 'a', encoding='utf-8')
    data_val   = open(fileval, 'a', encoding='utf-8')
    count = 0
    for content in contents:
        if len(content)>=limits:
            content = content.strip()
            count = count + 1
            if count<=lens:
                data_train.write(labels+'\t'+content+'\n')
            elif lens<count and count<=lens+1000:
                data_test.write(labels+'\t'+content+'\n')
            elif lens+1000<count and count<lens+1201:
                data_val.write(labels+'\t'+content+'\n')
            else:
                pass

In [28]:
limits = 145  #抽取超过特定长度的文本 
lens  =  5000 #每个样本抽取6000个样本
#AllTexts = [technology,sprots,society,military,international,house,home,car,finance,entertainment]
#labels   = ['技术','体育','社会','军事','国际','房产','家庭','汽车','金融','娱乐']
AllTexts = [international,house,home,car,finance,entertainment,technology,society,sprots]
labels   = ['国际','房产','家庭','汽车','金融', '娱乐','技术','社会','体育']
filetrain = './newdata/cnews_train.txt'
filetest  = './newdata/cnews_test.txt'
fileval   = './newdata/cnews_val.txt'
for i in range(0,len(AllTexts)):
    Train_Test_Validation_Data_Split(filetrain,filetest,fileval,AllTexts[i],labels[i],limits,lens)

### 1.3判断文件中的一行是不是为空，为空直接删掉

In [34]:
def del_blank(file1,file2):
    file1 = open(file1, 'r', encoding='utf-8') # 要去掉空行的文件 
    file2 = open(file2, 'w', encoding='utf-8') # 生成没有空行的文件
    try:
        for line in file1.readlines():
            if line == '\n':
                line = line.strip("\n")
            file2.write(line)
    finally:
        file1.close()
        file2.close()

In [ ]:
# 原始数据
filetrain = './newdata/cnews_train.txt'
filetest  = './newdata/cnews_test.txt'
fileval   = './newdata/cnews_val.txt'

# 清理空行后的数据
filetrain1 = './newdata/cnews.train.txt'
filetest1  = './newdata/cnews.test.txt'
fileval1   = './newdata/cnews.val.txt'

del_blank(filetrain,filetrain1)
del_blank(filetest,filetest1)
del_blank(fileval,fileval1)

### 1.4判断文件中的一行是不是为空，为空直接删掉；同时将样本乱序

In [29]:
import random
def delandrandom(file1,file2):
    contents = []
    file1 = open(file1, 'r', encoding='utf-8') # 要去掉空行的文件 
    file2 = open(file2, 'w', encoding='utf-8') # 生成没有空行的文件
    try:
        for line in file1.readlines():
            if line == '\n':
                line = line.strip("\n") #如果是空行直接删掉空行
                file2.write(line)
            else:
                contents.append(line)
            
        
        random.shuffle(contents)
        for content in contents:
            file2.write(content)
        
    finally:
        file1.close()
        file2.close()

In [30]:
# 原始数据
filetrain = './newdata/cnews_train.txt'
filetest  = './newdata/cnews_test.txt'
fileval   = './newdata/cnews_val.txt'

# 清理空行后的数据
filetrain1 = './newdata/random/cnews.train.txt'
filetest1  = './newdata/random/cnews.test.txt'
fileval1   = './newdata/random/cnews.val.txt'

delandrandom(filetrain,filetrain1)
delandrandom(filetest,filetest1)
delandrandom(fileval,fileval1)

### 基于样本收集词汇表和定制词嵌入矩阵###

In [100]:
from collections import Counter
import tensorflow.contrib.keras as kr
import numpy as np
import codecs
import re
import jieba

def read_file(filename):
    re_han = re.compile(u"([\u4E00-\u9FD5a-zA-Z0-9+#&\._%]+)")  # the method of cutting text by punctuation
    contents, labels = [], []
    with codecs.open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                line = line.rstrip()
                assert len(line.split('\t')) == 2
                label, content = line.split('\t')
                labels.append(label)
                blocks = re_han.split(content)
                word = []
                for blk in blocks:
                    if re_han.match(blk):
                        word.extend(jieba.lcut(blk))
                contents.append(word)
            except:
                pass
    return labels, contents
# labels[322] = '体育'
# np.array(contents[322]) = ['罗伊吃', '消炎药', '对抗', '膝伤', '开拓者',......]

In [101]:
def built_vocab_vector(filenames,stopword_path,vector_word_path,vocab_word_path,vector_word_npz_file,voc_size=10000):
    '''
    去停用词，得到前9999个词，获取对应的词 以及 词向量
    :param filenames:
    :param voc_size:
    :return:
    '''
    stopword = open(stopword_path, 'r', encoding='utf-8')
    stop = [key.strip(' \n') for key in stopword]
    all_data = []
    j = 1
    embeddings = np.zeros([10000, 100])  # 9656 是有效样本长度大小，仅仅需要更改这个即可，不许更改voc_size
    for filename in filenames:
        labels, content = read_file(filename)
        for eachline in content:
            line =[]
            for i in range(len(eachline)):
                if str(eachline[i]) not in stop:#去停用词
                    line.append(eachline[i])
            all_data.extend(line)
    counter = Counter(all_data)
    count_paris = counter.most_common(voc_size-1)
    word, _ = list(zip(*count_paris))
    f = codecs.open(vector_word_path, 'r', encoding='utf-8')
    vocab_word = open(vocab_word_path, 'w', encoding='utf-8')
    for ealine in f:
        item = ealine.split(' ')
        key = item[0]
        vec = np.array(item[1:], dtype='float32')
        if key in word:
            embeddings[j] = np.array(vec)
            vocab_word.write(key.strip('\r') + '\n')
            j += 1
    
    print("embeddings 填充行数是:",j)
    np.savez_compressed(vector_word_npz_file, embeddings=embeddings)

In [57]:
# 清理空行后的数据
train_filename = './newdata/random/cnews.train.txt'
test_filename  = './newdata/random/cnews.test.txt'
val_filename   = './newdata/random/cnews.val.txt'

In [102]:
# 清理空行后的数据
train_filename = './cnews/cnews.train.txt'
test_filename  = './cnews/cnews.test.txt'
val_filename   = './cnews/cnews.val.txt'

In [107]:
filenames = [train_filename,test_filename,val_filename] # input is one list

In [108]:
stopword_path    = './cnews/stopwords.txt'
vector_word_path = './cnews/vector_word.txt'
vocab_word_path  =  './cnews/vocab_word.txt'
vector_word_npz_file = './cnews/vector_word.npz'

In [109]:
import time
t1 = time.time()
built_vocab_vector(filenames,stopword_path,vector_word_path,vocab_word_path,vector_word_npz_file)
t2 = time.time()
print("运行时间%0.2f:"%((t2 - t1)/60),'mins')

TypeError: built_vocab_vector() got multiple values for argument 'voc_size'